In [0]:
import os
import keras

os.environ['CUDA_VISIBLE_DEVICES']='0'

In [0]:
train,test=keras.datasets.cifar10.load_data()

In [0]:
def preproc_x(x,flatten=True):
  x=x/255.
  if flatten:
    x=x.reshape((len(x),-1))
  return x

def preproc_y(y,num_classes=10):
  if y.shape[-1]==1:
    y=keras.utils.to_categorical(y,num_classes)
  return y

In [0]:
x_train,y_train=train
x_test,y_test=test

x_train=preproc_x(x_train)
x_test=preproc_x(x_test)

y_train=preproc_y(y_train)
y_test=preproc_y(y_test)

In [0]:
from keras.layers import BatchNormalization

def build_mlp(input_shape,output_units=10,num_neurons=[512,256,128]):
  input_layer=keras.layers.Input(input_shape)

  for i,n_units in enumerate(num_neurons):
    if i==0:
      x=keras.layers.Dense(units=n_units,activation='relu',name='hidden_layer'+str(i+1))(input_layer)
      x=BatchNormalization()(x)
    else:
      x=keras.layers.Dense(units=n_units,activation='relu',name='hidden_layer'+str(i+1))(x)
      x=BatchNormalization()(x)
  out=keras.layers.Dense(units=output_units,activation='softmax',name='output')(x)

  model=keras.models.Model(inputs=[input_layer],outputs=[out])
  return model


In [0]:
LEARNING_RATE=1e-3
EPOCHS=50
BATCH_SIZE=[2, 16, 32, 128, 256]
MOMENTUM=0.95

In [0]:
model=build_mlp(input_shape=x_train.shape[1:])
model.summary()
optimizer=keras.optimizers.SGD(lr=LEARNING_RATE,nesterov=True,momentum=MOMENTUM)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=optimizer)
for i,bs in enumerate(BATCH_SIZE):
  model.fit(x_train,y_train,epochs=EPOCHS,batch_size=bs,validation_data=(x_test,y_test),shuffle=True)

  train_loss[i] = model.history.history["loss"]
  valid_loss[i] = model.history.history["val_loss"]
  train_acc[i] = model.history.history["acc"]
  valid_acc[i] = model.history.history["val_acc"]

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

for j in range(i):
  plt.plot(range(len(train_loss)), train_loss[j], label="train loss")
  plt.plot(range(len(valid_loss)), valid_loss[j], label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

for k in range(i):
  plt.plot(range(len(train_acc)), train_acc[k], label="train accuracy")
  plt.plot(range(len(valid_acc)), valid_acc[k], label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()